<img src="img/logo_p8.png">

# Importation des librairies

In [1]:
import numpy as np
import pandas as pd
from mylib.fonction_exploration import exploration1, exploration2, exploration3, exploration4
from mylib.fonction_pk import test_pk
from mylib.fonction_merge import df_merge
import warnings
warnings.filterwarnings('ignore')
#!pip install fuzzywuzzy
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

print("Version numpy : " + np.__version__)
print("Version pandas : " + pd.__version__)

bold = "\033[1m"
red = "\033[31m"
end = "\033[0;0m"

chemin1 = 'src/BasicAndSafelyManagedDrinkingWaterServices.csv'
chemin2 = 'src/MortalityRateAttributedToWater.csv'
chemin3 = 'src/PoliticalStability.csv'
chemin4 = 'src/Population.csv'
chemin5 = 'src/RegionCountry.csv'

Version numpy : 1.23.5
Version pandas : 1.4.4


# Chargement des fichiers

In [2]:
water = pd.read_csv(chemin1)
mortal = pd.read_csv(chemin2)
stab = pd.read_csv(chemin3)
pop = pd.read_csv(chemin4)
country = pd.read_csv(chemin5)

# **Countries**

## Observations uniques

In [3]:
for i in country.columns:
    print(bold + i + end, "\n", country[i].unique()[0:25], "\n")

REGION (DISPLAY) 
 ['Europe' 'Western Pacific' 'Eastern Mediterranean' 'South-East Asia'
 'Africa' 'Americas'] 

COUNTRY (DISPLAY) 
 ['Albania' 'Andorra' 'Armenia' 'Australia' 'Austria' 'Azerbaijan'
 'Bahrain' 'Bangladesh' 'Belarus' 'Belgium' 'Bhutan'
 'Bosnia and Herzegovina' 'Botswana' 'Brazil' 'Bulgaria' 'Cambodia'
 'Canada' 'Chile' 'China' 'Colombia' 'Congo' 'Costa Rica' 'Croatia'
 'Cyprus' 'Czechia'] 



## Exploration

In [4]:
# La fonction est visible dans le fichier mylib
exploration1(country)

,REGION (DISPLAY),COUNTRY (DISPLAY)
0,Europe,Albania
1,Europe,Andorra
2,Europe,Armenia
,REGION (DISPLAY),COUNTRY (DISPLAY)
191,Western Pacific,Viet Nam
192,Eastern Mediterranean,Yemen
193,Africa,Zimbabwe
,REGION (DISPLAY),COUNTRY (DISPLAY)
50,Europe,Italy
65,Europe,Monaco


In [5]:
# La fonction est visible dans le fichier mylib
exploration2(country)

,Type des données,Nombre de valeurs totales,Nombre de NaN,NaN (%),Nombre de valeurs uniques,Valeurs uniques (%)
REGION (DISPLAY),object,194,0,0.0,6,3.092784
COUNTRY (DISPLAY),object,194,0,0.0,194,100.0


In [6]:
# La fonction est visible dans le fichier mylib
exploration3(country)


---------------------------------------
Nombre de lignes et de colonnes (shape)
--------------------------------------- 

(194, 2) 

--------------------------------------
Affichage des NaN (isna().any(axis=1))
-------------------------------------- 

Empty DataFrame
Columns: [REGION (DISPLAY), COUNTRY (DISPLAY)]
Index: [] 

---------------------------------------
Nombre de doublons (duplicated().sum())
--------------------------------------- 

0 

-----------------------------------------------------
Affichage des doublons (df[df.duplicated()].head(10))
----------------------------------------------------- 

Empty DataFrame
Columns: [REGION (DISPLAY), COUNTRY (DISPLAY)]
Index: [] 



In [7]:
# La fonction est visible dans le fichier mylib
exploration4(country)

,count,unique,top,freq
REGION (DISPLAY),194,6,Europe,53
COUNTRY (DISPLAY),194,194,Albania,1


In [8]:
# Renommage de colonne
country.rename(columns={'REGION (DISPLAY)': 'Region', 'COUNTRY (DISPLAY)': 'Country' }, inplace=True)

In [9]:
country.sample()

,Region,Country
121,Americas,Barbados


## Vérification des incohérences dans les noms des pays

### Pays absents du DataFrame 'country'

In [10]:
# Crée un dictionnaire pour stocker les colonnes 'Country' de chaque DataFrame
dataframes = {'country': country['Country'],
              'pop': pop['Country'],
              'water': water['Country'],
              'stab': stab['Country'],
              'mortal': mortal['Country']}

# Vérifie les incohérences par rapport au DataFrame 'country'
inconsistencies = {}

# Parcourt chaque DataFrame et compare les pays avec le DataFrame 'country'
for name, df_countries in dataframes.items():
    inconsistencies[name] = df_countries[~df_countries.isin(dataframes['country'])]

# Affiche les incohérences sous forme de valeurs uniques
for name, inconsistency in inconsistencies.items():
    if not inconsistency.empty:
        print(bold + red + f"{inconsistency.nunique()} incohérences dans le DataFrame '{name}':" + end)
        print(inconsistency.unique(), '\n')
    else:
        print(bold + f"Il n'y a pas d'incohérences dans le DataFrame '{name}'\n" + end)

Il n'y a pas d'incohérences dans le DataFrame 'country'

46 incohérences dans le DataFrame 'pop':
['American Samoa' 'Anguilla' 'Aruba' 'Bermuda'
 'Bonaire, Sint Eustatius and Saba' 'British Virgin Islands'
 'Cayman Islands' 'Channel Islands' 'China, Hong Kong SAR'
 'China, Macao SAR' 'China, mainland' 'China, Taiwan Province of'
 'Curaçao' 'Falkland Islands (Malvinas)' 'Faroe Islands' 'French Guyana'
 'French Polynesia' 'Gibraltar' 'Greenland' 'Guadeloupe' 'Guam' 'Holy See'
 'Isle of Man' 'Liechtenstein' 'Martinique' 'Mayotte' 'Montserrat'
 'Netherlands Antilles (former)' 'New Caledonia' 'North Macedonia'
 'Northern Mariana Islands' 'Palestine' 'Puerto Rico' 'Réunion'
 'Saint Barthélemy' 'Saint Helena, Ascension and Tristan da Cunha'
 'Saint Pierre and Miquelon' 'Saint-Martin (French part)'
 'Serbia and Montenegro' 'Sint Maarten  (Dutch part)' 'Sudan (former)'
 'Tokelau' 'Turks and Caicos Islands' 'United States Virgin Islands'
 'Wallis and Futuna Islands' 'Western Sahara'] 

Il n'y a 

### Pays du DataFrame 'country' absents des autres DataFrames

In [11]:
# Crée un dictionnaire pour stocker les colonnes 'Country' de chaque DataFrame
dataframes = {'country': country['Country'],
              'pop': pop['Country'],
              'water': water['Country'],
              'stab': stab['Country'],
              'mortal': mortal['Country']}

# Vérifie si les pays du DataFrame 'country' sont tous présents dans les autres DataFrames
country_countries = set(dataframes['country'])
missing_countries = {}

# Parcourt chaque DataFrame et vérifie si les pays du DataFrame 'country' sont présents
for name, df_countries in dataframes.items():
    if name != 'country':
        missing_countries[name] = list(country_countries - set(df_countries))

# Affiche les pays manquants dans les autres DataFrames
for name, missing in missing_countries.items():
    if missing:
        if len(missing) == 1:
            print(bold + red + f"{len(missing)} pays manquant dans le DataFrame '{name}':" + end)
            print(', '.join(missing), '\n')
        else:
            print(bold + red + f"{len(missing)} pays manquants dans le DataFrame '{name}':" + end)
            print(', '.join(missing), '\n')

1 pays manquant dans le DataFrame 'pop':
Republic of North Macedonia 

4 pays manquants dans le DataFrame 'stab':
China, San Marino, Republic of North Macedonia, Monaco 

11 pays manquants dans le DataFrame 'mortal':
San Marino, Cook Islands, Nauru, Saint Kitts and Nevis, Andorra, Palau, Marshall Islands, Tuvalu, Niue, Dominica, Monaco 



In [12]:
# Aperçu des intitulés, on sélectionne les observations "China" et "China, mainland" pour l'année 2016
china_pop_2016 = pop.loc[(pop['Country'].isin(['China', 'China, mainland'])) & (pop['Year'] == 2016)]
china_stab_2016 = stab.loc[(stab['Country'].isin(['China', 'China, mainland'])) & (stab['Year'] == 2016)]

# Affiche les observations correspondantes
china_pop_2016

,Country,Granularity,Year,Population
3866,China,Total,2016,1445523.929
3867,China,Male,2016,741481.549
3868,China,Female,2016,704042.370
3869,China,Rural,2016,612549.421
3870,China,Urban,2016,822422.660
4151,"China, mainland",Total,2016,1414049.351
4152,"China, mainland",Male,2016,726052.413
4153,"China, mainland",Female,2016,687996.940
4154,"China, mainland",Rural,2016,607210.874
4155,"China, mainland",Urban,2016,796289.491


In [13]:
china_stab_2016

,Country,Year,Political_Stability,Granularity
714,"China, mainland",2016,-0.5,Total


### Correction des incohérences

#### Pop

In [14]:
# Supprime les observations pour le pays "China, mainland" pour éviter les doublons, on garde "China"
pop = pop.drop(pop[pop['Country'] == 'China, mainland'].index)

# Vérification
china_pop_2016 = pop.loc[(pop['Country'].isin(['China', 'China, mainland'])) & (pop['Year'] == 2016)]
china_pop_2016

,Country,Granularity,Year,Population
3866,China,Total,2016,1445523.929
3867,China,Male,2016,741481.549
3868,China,Female,2016,704042.370
3869,China,Rural,2016,612549.421
3870,China,Urban,2016,822422.660


In [15]:
# Crée un dictionnaire de correspondance entre les anciens noms de pays et les nouveaux noms
country_mapping = {
    'China, Hong Kong SAR': 'Hong Kong',
    'Serbia and Montenegro': 'Serbia',
    'China, Macao SAR': 'Macao',
    'China, Taiwan Province of': 'Taiwan',
    'North Macedonia': 'Republic of North Macedonia'
}

# Remplace les anciens noms de pays par les nouveaux noms dans la colonne 'Country' du DataFrame pop
pop['Country'] = pop['Country'].replace(country_mapping)

# Vérifie les modifications
#print(pop['Country'].unique())

#### Stab

In [16]:
# Crée un dictionnaire de correspondance entre les anciens noms de pays et les nouveaux noms
country_mapping = {
    'China, Hong Kong SAR': 'Hong Kong',
    'China, mainland': 'China',
    'China, Macao SAR': 'Macao',
    'China, Taiwan Province of': 'Taiwan',
    'North Macedonia': 'Republic of North Macedonia'
}

# Remplace les anciens noms de pays par les nouveaux noms dans la colonne 'Country' du DataFrame pop
stab['Country'] = stab['Country'].replace(country_mapping)

#### Country

In [17]:
# Nombre de pays avant concaténation
country.shape

(194, 2)

In [18]:
# Obtient la liste des pays présents dans pop mais absents de country
missing_countries = list(set(pop['Country']) - set(country['Country']))

# Crée un DataFrame avec les pays manquants
missing_rows = pop[pop['Country'].isin(missing_countries)]

# Supprime les doublons des lignes manquantes
missing_rows = missing_rows.drop_duplicates(subset='Country')

# Ajoute les lignes manquantes à country
country = pd.concat([country, missing_rows], ignore_index=True)
country.drop(columns=['Granularity', 'Year', 'Population'], inplace=True)
country.sample()

,Region,Country
25,Africa,Côte d'Ivoire


In [19]:
# Nombre de pays après concaténation
country.shape

(237, 2)

In [20]:
# Liste les valeurs NaN dans la colonne 'Region'
nan_values = country.loc[country['Region'].isnull(), 'Country'].unique()

# Affiche les pays uniques ayant des valeurs NaN dans la colonne 'Region'
print(bold + red + f"{len(nan_values)} pays avec des valeurs NaN dans la colonne 'Region':" + end)
print(', '.join(nan_values))

43 pays avec des valeurs NaN dans la colonne 'Region':
American Samoa, Anguilla, Aruba, Bermuda, Bonaire, Sint Eustatius and Saba, British Virgin Islands, Cayman Islands, Channel Islands, Hong Kong, Macao, Taiwan, Curaçao, Falkland Islands (Malvinas), Faroe Islands, French Guyana, French Polynesia, Gibraltar, Greenland, Guadeloupe, Guam, Holy See, Isle of Man, Liechtenstein, Martinique, Mayotte, Montserrat, Netherlands Antilles (former), New Caledonia, Northern Mariana Islands, Palestine, Puerto Rico, Réunion, Saint Barthélemy, Saint Helena, Ascension and Tristan da Cunha, Saint Pierre and Miquelon, Saint-Martin (French part), Sint Maarten  (Dutch part), Sudan (former), Tokelau, Turks and Caicos Islands, United States Virgin Islands, Wallis and Futuna Islands, Western Sahara


In [21]:
country['Region'].unique()

array(['Europe', 'Western Pacific', 'Eastern Mediterranean',
       'South-East Asia', 'Africa', 'Americas', nan], dtype=object)

In [22]:
# Définit le mapping pour compléter les valeurs NaN en fonction des pays
region_mapping = {'American Samoa': 'Americas',
                  'Anguilla': 'Americas',
                  'Aruba': 'Americas',
                  'Bermuda': 'Americas',
                  'Bonaire, Sint Eustatius and Saba': 'Americas',
                  'British Virgin Islands': 'Americas',
                  'Cayman Islands': 'Americas',
                  'Channel Islands': 'Europe',
                  'Hong Kong': 'South-East Asia',
                  'Macao': 'South-East Asia',
                  'Taiwan': 'South-East Asia',
                  'Curaçao': 'Americas',
                  'Falkland Islands (Malvinas)': 'Americas',
                  'Faroe Islands': 'Europe',
                  'French Guyana': 'Americas',
                  'French Polynesia': 'Western Pacific',
                  'Gibraltar': 'Europe',
                  'Greenland': 'Americas',
                  'Guadeloupe': 'Americas',
                  'Guam': 'Western Pacific',
                  'Holy See': 'Europe',
                  'Isle of Man': 'Europe',
                  'Liechtenstein': 'Europe',
                  'Martinique': 'Americas',
                  'Mayotte': 'Africa',
                  'Montserrat': 'Americas',
                  'Netherlands Antilles (former)': 'Americas',
                  'New Caledonia': 'Western Pacific',
                  'Northern Mariana Islands': 'Western Pacific',
                  'Palestine': 'Eastern Mediterranean',
                  'Puerto Rico': 'Americas',
                  'Réunion': 'Africa',
                  'Saint Barthélemy': 'Americas',
                  'Saint Helena, Ascension and Tristan da Cunha': 'Africa',
                  'Saint Pierre and Miquelon': 'Americas',
                  'Saint-Martin (French part)': 'Americas',
                  'Sint Maarten  (Dutch part)': 'Americas',
                  'Sudan (former)': 'Africa',
                  'Tokelau': 'Western Pacific',
                  'Turks and Caicos Islands': 'Americas',
                  'United States Virgin Islands': 'Americas',
                  'Wallis and Futuna Islands': 'Western Pacific',
                  'Western Sahara': 'Africa'}

# Complète les valeurs NaN dans la colonne 'Region' en fonction du mapping
country['Region'] = country['Region'].fillna(country['Country'].map(region_mapping))

### Vérification après nettoyage de la variable 'Country'

In [23]:
# Liste les valeurs NaN dans la colonne 'Region'
nan_values = country.loc[country['Region'].isnull(), 'Country'].unique()

# Affiche les pays ayant des valeurs NaN dans la colonne 'Region'
print(bold + f"{len(nan_values)} pays avec des valeurs NaN dans la colonne 'Region'" + end)
print(', '.join(nan_values))

0 pays avec des valeurs NaN dans la colonne 'Region'



In [24]:
# Crée un dictionnaire pour stocker les colonnes 'Country' de chaque DataFrame
dataframes = {'country': country['Country'],
              'pop': pop['Country'],
              'water': water['Country'],
              'stab': stab['Country'],
              'mortal': mortal['Country']}

# Vérifie les incohérences par rapport au DataFrame 'country'
inconsistencies = {}

# Parcourt chaque DataFrame et compare les pays avec le DataFrame 'country'
for name, df_countries in dataframes.items():
    inconsistencies[name] = df_countries[~df_countries.isin(dataframes['country'])]

# Affiche les incohérences sous forme de valeurs uniques
for name, inconsistency in inconsistencies.items():
    if not inconsistency.empty:
        print(bold + red + f"{inconsistency.nunique()} incohérences dans le DataFrame '{name}':")
        print(inconsistency.unique(), '\n')
    else:
        print(bold + f"Il n'y a pas d'incohérences dans le DataFrame '{name}'" + end)

Il n'y a pas d'incohérences dans le DataFrame 'country'
Il n'y a pas d'incohérences dans le DataFrame 'pop'
Il n'y a pas d'incohérences dans le DataFrame 'water'
Il n'y a pas d'incohérences dans le DataFrame 'stab'
Il n'y a pas d'incohérences dans le DataFrame 'mortal'


In [25]:
# Crée un dictionnaire pour stocker les colonnes 'Country' de chaque DataFrame
dataframes = {'country': country['Country'],
              'pop': pop['Country'],
              'water': water['Country'],
              'stab': stab['Country'],
              'mortal': mortal['Country']}

# Vérifie si les pays du DataFrame 'country' sont tous présents dans les autres DataFrames
country_countries = set(dataframes['country'])
missing_countries = {}

# Parcourt chaque DataFrame et vérifie si les pays du DataFrame 'country' sont présents
for name, df_countries in dataframes.items():
    if name != 'country':
        missing_countries[name] = list(country_countries - set(df_countries))

# Affiche les pays manquants dans les autres DataFrames
for name, missing in missing_countries.items():
    if missing:
        print(bold + red + f"Pays manquants dans le DataFrame '{name}':" + end)
        print(', '.join(missing), '\n')

Pays manquants dans le DataFrame 'water':
Turks and Caicos Islands, Gibraltar, Hong Kong, Saint-Martin (French part), United States Virgin Islands, Anguilla, American Samoa, Aruba, Puerto Rico, Netherlands Antilles (former), Sint Maarten  (Dutch part), Cayman Islands, Guadeloupe, Northern Mariana Islands, Tokelau, Faroe Islands, Saint Pierre and Miquelon, Wallis and Futuna Islands, Bonaire, Sint Eustatius and Saba, Saint Helena, Ascension and Tristan da Cunha, Sudan (former), Isle of Man, Mayotte, Martinique, Guam, Taiwan, Saint Barthélemy, Curaçao, Réunion, Channel Islands, Falkland Islands (Malvinas), Liechtenstein, Macao, Montserrat, French Polynesia, New Caledonia, Greenland, British Virgin Islands, Bermuda, Palestine, Holy See, Western Sahara, French Guyana 

Pays manquants dans le DataFrame 'stab':
Turks and Caicos Islands, Gibraltar, Saint-Martin (French part), United States Virgin Islands, Anguilla, Aruba, Netherlands Antilles (former), San Marino, Sint Maarten  (Dutch part), C

**=> Ce résultat est normal car nous avons complété le DataFrame 'country'**

# **Population**

## Observations uniques

In [26]:
for i in pop.columns:
    print(bold + i + end, "\n", pop[i].unique()[0:25], "\n")

Country 
 ['Afghanistan' 'Albania' 'Algeria' 'American Samoa' 'Andorra' 'Angola'
 'Anguilla' 'Antigua and Barbuda' 'Argentina' 'Armenia' 'Aruba'
 'Australia' 'Austria' 'Azerbaijan' 'Bahamas' 'Bahrain' 'Bangladesh'
 'Barbados' 'Belarus' 'Belgium' 'Belize' 'Benin' 'Bermuda' 'Bhutan'
 'Bolivia (Plurinational State of)'] 

Granularity 
 ['Total' 'Male' 'Female' 'Rural' 'Urban'] 

Year 
 [2000 2001 2002 2003 2004 2005 2006 2007 2008 2009 2010 2011 2012 2013
 2014 2015 2016 2017 2018] 

Population 
 [20779.953 10689.508 10090.449 15657.474  4436.282 21606.988 11117.754
 10489.238 16318.324  4648.139 22600.77  11642.106 10958.668 17086.91
  4893.013 23680.871 12214.634 11466.237 17909.063  5155.788 24726.684
 12763.726 11962.963 18692.107  5426.872] 



## Exploration

In [27]:
# La fonction est visible dans le fichier mylib
exploration1(pop)

,Country,Granularity,Year,Population
0,Afghanistan,Total,2000,20779.953000
1,Afghanistan,Male,2000,10689.508000
2,Afghanistan,Female,2000,10090.449000
,Country,Granularity,Year,Population
20911,Zimbabwe,Female,2018,7559.693000
20912,Zimbabwe,Rural,2018,11465.748000
20913,Zimbabwe,Urban,2018,5447.513000
,Country,Granularity,Year,Population
6805,Finland,Male,2018,2721.488000
8956,India,Male,2015,681223.332000


In [28]:
# La fonction est visible dans le fichier mylib
exploration2(pop)

,Type des données,Nombre de valeurs totales,Nombre de NaN,NaN (%),Nombre de valeurs uniques,Valeurs uniques (%)
Country,object,20819,0,0.0,237,1.138383
Granularity,object,20819,0,0.0,5,0.024017
Year,int64,20819,0,0.0,19,0.091263
Population,float64,20819,0,0.0,20420,98.083481


In [29]:
# La fonction est visible dans le fichier mylib
exploration3(pop)


---------------------------------------
Nombre de lignes et de colonnes (shape)
--------------------------------------- 

(20819, 4) 

--------------------------------------
Affichage des NaN (isna().any(axis=1))
-------------------------------------- 

Empty DataFrame
Columns: [Country, Granularity, Year, Population]
Index: [] 

---------------------------------------
Nombre de doublons (duplicated().sum())
--------------------------------------- 

0 

-----------------------------------------------------
Affichage des doublons (df[df.duplicated()].head(10))
----------------------------------------------------- 

Empty DataFrame
Columns: [Country, Granularity, Year, Population]
Index: [] 



In [30]:
# La fonction est visible dans le fichier mylib
exploration4(pop)

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
Country,20819,237,Afghanistan,95,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Granularity,20819,5,Total,4411,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Year,20819.0,NaN,NaN,NaN,2009.046928,5.479204,2000.0,2004.0,2009.0,2014.0,2018.0
Population,20819.0,NaN,NaN,NaN,18917.216851,82526.486744,0.0,341.461,2981.269,10928.1945,1459377.612


In [31]:
# Mise à l'échelle de la population pour affichage réel et renommage de colonne
pop['Population'] = pop['Population'] * 1000
pop.rename(columns={'Granularity': 'Granularity population (gender/urban/rural)'}, inplace=True)
pop.sample()

,Country,Granularity population (gender/urban/rural),Year,Population
1142,Azerbaijan,Male,2004,4156206.0


## Vérifions l'exactitude du nom des pays

In [32]:
pop_countries = pop['Country'].unique()
country_countries = country['Country'].unique()

# Compare les valeurs uniques des deux DataFrames
common_countries = set(pop_countries) & set(country_countries)
unique_countries_in_pop = set(pop_countries) - set(country_countries)
unique_countries_in_country = set(country_countries) - set(pop_countries)

# Affiche les résultats
print(bold + "Pays communs aux deux DataFrames :")
print("Nombre :", len(common_countries), end)
print(common_countries)
print(bold + "\nPays uniquement dans le DataFrame pop :")
print("Nombre :", len(unique_countries_in_pop), end)
print(unique_countries_in_pop)
print(bold + "\nPays uniquement dans le DataFrame country :")
print("Nombre :", len(unique_countries_in_country), end)
print(unique_countries_in_country)

Pays communs aux deux DataFrames :
Nombre : 237 
{'Belarus', 'Kyrgyzstan', 'France', 'Republic of North Macedonia', 'Fiji', 'Honduras', 'Austria', 'Portugal', 'Liberia', 'Armenia', 'Estonia', 'Georgia', 'Russian Federation', 'Anguilla', 'Malawi', 'Sri Lanka', 'Syrian Arab Republic', 'Yemen', 'Italy', 'Albania', 'China', 'United States of America', 'Puerto Rico', 'San Marino', 'Libya', 'Equatorial Guinea', 'Philippines', 'Guadeloupe', 'Latvia', 'Niue', 'Qatar', 'Samoa', 'Faroe Islands', 'Saint Pierre and Miquelon', 'Bonaire, Sint Eustatius and Saba', 'Zambia', 'Cyprus', 'Lithuania', 'Lebanon', 'Iceland', 'Viet Nam', 'Mayotte', 'Martinique', 'Indonesia', 'Thailand', 'Curaçao', 'Central African Republic', 'Djibouti', 'Channel Islands', 'Réunion', 'Senegal', 'Montserrat', 'Spain', 'French Polynesia', 'Solomon Islands', 'Denmark', 'Ecuador', 'Greece', 'Holy See', 'Namibia', 'Monaco', 'Montenegro', 'Bulgaria', "Democratic People's Republic of Korea", 'Turks and Caicos Islands', 'Lesotho', 'N

**=> Pas de corrections à faire, pour la jointure il est important que tous les pays du DataFrame 'pop' soient présents dans le DataFrame 'country'**

In [33]:
# Obtiens les valeurs uniques de la colonne 'Country' dans le premier DataFrame (pop)
pop_countries = pop['Country'].unique()

# Obtiens les valeurs uniques de la colonne 'Country' dans le deuxième DataFrame (country)
country_countries = country['Country'].unique()

# Définit un seuil de correspondance (score minimal pour considérer les pays comme correspondants)
threshold = 85

# Compare les noms de pays entre les deux DataFrames
matching_countries = []
for country_pop in pop_countries:
    best_match = process.extractOne(country_pop, country_countries, scorer=fuzz.token_set_ratio)
    if best_match[1] >= threshold and country_pop != best_match[0]:
        matching_countries.append((country_pop, best_match[0]))

# Affiche les résultats
print(bold + "Correspondances entre les noms de pays (excluant les identiques) :" + end)
for country_pair in matching_countries:
    print("Pop : {}, Country : {}".format(country_pair[0], country_pair[1]))

Correspondances entre les noms de pays (excluant les identiques) :
Pop : American Samoa, Country : Samoa
Pop : Democratic Republic of the Congo, Country : Congo
Pop : French Guyana, Country : Guyana
Pop : Guinea, Country : Equatorial Guinea
Pop : Guinea-Bissau, Country : Guinea
Pop : Netherlands Antilles (former), Country : Netherlands
Pop : Papua New Guinea, Country : Guinea
Pop : Republic of Korea, Country : Democratic People's Republic of Korea
Pop : Sudan, Country : South Sudan
Pop : Sudan (former), Country : Sudan
Pop : United Kingdom of Great Britain and Northern Ireland, Country : Ireland


**=> Il s'agit bien de pays différents, il n'y a pas de correction à faire**

In [34]:
# shape avant merging
pop.shape

(20819, 4)

In [35]:
# la fonction est visible dans le fichier mylib
df_merge(pop, country, ['Country'])

Le DataFrame contient 20819 lignes

La proportion de 'both' est de 100.0 % soit 20819 lignes
La proportion de 'left_only' est de 0.0 % 0 lignes
La proportion de 'right_only' est de 0.0 % 0 lignes


In [36]:
pop = pop.merge(country, on=['Country'], how='inner')
pop.sample()

,Country,Granularity population (gender/urban/rural),Year,Population,Region
11522,Mali,Rural,2011,9822006.0,Africa


In [37]:
# shape après merging
pop.shape

(20819, 5)

# **Water**

## Observations uniques

In [38]:
for i in water.columns:
    print(bold + i + end, "\n", water[i].unique()[0:25], "\n")

Year 
 [2000 2001 2002 2003 2004 2005 2006 2007 2008 2009 2010 2011 2012 2013
 2014 2015 2016 2017] 

Country 
 ['Afghanistan' 'Albania' 'Algeria' 'Andorra' 'Angola'
 'Antigua and Barbuda' 'Argentina' 'Armenia' 'Australia' 'Austria'
 'Azerbaijan' 'Bahamas' 'Bahrain' 'Bangladesh' 'Barbados' 'Belarus'
 'Belgium' 'Belize' 'Benin' 'Bhutan' 'Bolivia (Plurinational State of)'
 'Bosnia and Herzegovina' 'Botswana' 'Brazil' 'Brunei Darussalam'] 

Granularity 
 ['Rural' 'Total' 'Urban'] 

Population using at least basic drinking-water services (%) 
 [ 21.61913  27.7719   49.48745  81.78472  87.86662  96.35529  83.32201
  89.83726  94.19543 100.       21.15264  41.14431  61.06653       nan
  98.25377  76.40468  96.21301  98.62579  88.56082  95.13351  98.72488
  99.05666  99.7123   99.835    54.93393] 

Population using safely managed drinking-water services (%) 
 [     nan 49.29324 90.64    29.59756 98.16657 97.71394 49.67768 98.79928
 59.41035 55.9067  44.55809 80.62173 99.52319 21.73112 28.5512

## Exploration

In [39]:
# La fonction est visible dans le fichier mylib
exploration1(water)

,Year,Country,Granularity,Population using at least basic drinking-water services (%),Population using safely managed drinking-water services (%)
0,2000,Afghanistan,Rural,21.619130,nan
1,2000,Afghanistan,Total,27.771900,nan
2,2000,Afghanistan,Urban,49.487450,nan
,Year,Country,Granularity,Population using at least basic drinking-water services (%),Population using safely managed drinking-water services (%)
10473,2017,Zimbabwe,Rural,49.804760,nan
10474,2017,Zimbabwe,Total,64.051230,nan
10475,2017,Zimbabwe,Urban,93.997670,nan
,Year,Country,Granularity,Population using at least basic drinking-water services (%),Population using safely managed drinking-water services (%)
8632,2014,South Africa,Total,91.426200,nan
6341,2010,Togo,Urban,84.948230,nan


In [40]:
# La fonction est visible dans le fichier mylib
exploration2(water)

,Type des données,Nombre de valeurs totales,Nombre de NaN,NaN (%),Nombre de valeurs uniques,Valeurs uniques (%)
Year,int64,10476,0,0.0,18,0.171821
Country,object,10476,0,0.0,194,1.851852
Granularity,object,10476,0,0.0,3,0.028637
Population using at least basic drinking-water services (%),float64,9415,1061,10.127911,7704,73.539519
Population using safely managed drinking-water services (%),float64,3286,7190,68.633066,2877,27.462772


In [41]:
# La fonction est visible dans le fichier mylib
exploration3(water)


---------------------------------------
Nombre de lignes et de colonnes (shape)
--------------------------------------- 

(10476, 5) 

--------------------------------------
Affichage des NaN (isna().any(axis=1))
-------------------------------------- 

       Year      Country Granularity  \
0      2000  Afghanistan       Rural   
1      2000  Afghanistan       Total   
2      2000  Afghanistan       Urban   
3      2000      Albania       Rural   
5      2000      Albania       Urban   
...     ...          ...         ...   
10470  2017       Zambia       Rural   
10471  2017       Zambia       Total   
10473  2017     Zimbabwe       Rural   
10474  2017     Zimbabwe       Total   
10475  2017     Zimbabwe       Urban   

       Population using at least basic drinking-water services (%)  \
0                                               21.61913             
1                                               27.77190             
2                                               49.487

In [42]:
# La fonction est visible dans le fichier mylib
exploration4(water)

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
Year,10476.0,NaN,NaN,NaN,2008.5,5.188375,2000.0,2004.0,2008.5,2013.0,2017.0
Country,10476,194,Afghanistan,54,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Granularity,10476,3,Rural,3492,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Population using at least basic drinking-water services (%),9415.0,NaN,NaN,NaN,83.96212,19.968269,4.08262,75.928395,93.1154,98.95424,100.00001
Population using safely managed drinking-water services (%),3286.0,NaN,NaN,NaN,66.070856,30.383942,0.0,41.895583,73.966655,94.77664,100.0


In [43]:
# Renommage de colonne
water.rename(columns={'Granularity': 'Granularity water service'}, inplace=True)

In [44]:
water.sample()

,Year,Country,Granularity water service,Population using at least basic drinking-water services (%),Population using safely managed drinking-water services (%)
2880,2004,United Republic of Tanzania,Rural,21.87435,NaN


## Vérifions l'exactitude du nom des pays

In [45]:
water_countries = water['Country'].unique()
country_countries = country['Country'].unique()

# Compare les valeurs uniques des deux DataFrames
common_countries = set(water_countries) & set(country_countries)
unique_countries_in_water = set(water_countries) - set(country_countries)
unique_countries_in_country = set(country_countries) - set(water_countries)

# Affiche les résultats
print(bold + "Pays communs aux deux DataFrames :")
print("Nombre :", len(common_countries), end)
print(common_countries)
print(bold + "\nPays uniquement dans le DataFrame water :")
print("Nombre :", len(unique_countries_in_water), end)
print(unique_countries_in_water)
print(bold + "\nPays uniquement dans le DataFrame country :")
print("Nombre :", len(unique_countries_in_country), end)
print(unique_countries_in_country)

Pays communs aux deux DataFrames :
Nombre : 194 
{'Belarus', 'Kyrgyzstan', 'France', 'Republic of North Macedonia', 'Fiji', 'Honduras', 'Austria', 'Portugal', 'Liberia', 'Armenia', 'Estonia', 'Georgia', 'Russian Federation', 'Malawi', 'Sri Lanka', 'Syrian Arab Republic', 'Yemen', 'Italy', 'Albania', 'China', 'United States of America', 'San Marino', 'Libya', 'Equatorial Guinea', 'Philippines', 'Latvia', 'Niue', 'Qatar', 'Samoa', 'Zambia', 'Cyprus', 'Lithuania', 'Lebanon', 'Iceland', 'Viet Nam', 'Indonesia', 'Thailand', 'Central African Republic', 'Djibouti', 'Senegal', 'Spain', 'Solomon Islands', 'Denmark', 'Ecuador', 'Greece', 'Namibia', 'Monaco', 'Montenegro', 'Bulgaria', "Democratic People's Republic of Korea", 'Lesotho', 'Nigeria', 'Sierra Leone', 'Saint Lucia', 'Iraq', 'Cambodia', 'Mexico', 'Peru', 'Bahrain', 'Czechia', 'Pakistan', 'Suriname', 'Panama', 'Ukraine', 'Haiti', 'Guinea-Bissau', 'Oman', 'Nepal', 'Republic of Moldova', 'Comoros', 'Somalia', 'Japan', 'United Republic of T

**=> Pas de corrections à faire, pour la jointure il est important que tous les pays du DataFrame 'water' soient présents dans le DataFrame 'country'**

In [46]:
# Obtiens les valeurs uniques de la colonne 'Country' dans le premier DataFrame (pop)
water_countries = water['Country'].unique()

# Obtiens les valeurs uniques de la colonne 'Country' dans le deuxième DataFrame (country)
country_countries = country['Country'].unique()

# Définit un seuil de correspondance (score minimal pour considérer les pays comme correspondants)
threshold = 80

# Compare les noms de pays entre les deux DataFrames
matching_countries = []
for country_water in water_countries:
    best_match = process.extractOne(country_water, country_countries, scorer=fuzz.token_set_ratio)
    if best_match[1] >= threshold and country_water != best_match[0]:
        matching_countries.append((country_water, best_match[0]))

# Affiche les résultats
print(bold + "Correspondances entre les noms de pays (excluant les identiques) :" + end)
for country_pair in matching_countries:
    print("Pop : {}, Country : {}".format(country_pair[0], country_pair[1]))

Correspondances entre les noms de pays (excluant les identiques) :
Pop : Democratic Republic of the Congo, Country : Congo
Pop : Guinea, Country : Equatorial Guinea
Pop : Guinea-Bissau, Country : Guinea
Pop : Papua New Guinea, Country : Guinea
Pop : Republic of Korea, Country : Democratic People's Republic of Korea
Pop : Sudan, Country : South Sudan
Pop : United Kingdom of Great Britain and Northern Ireland, Country : Ireland


**=> Il s'agit bien de pays différents, il n'y a pas de correction à faire**

# **Mortality due to water**

## Observations uniques

In [47]:
for i in mortal.columns:
    print(bold + i + end, "\n", mortal[i].unique()[0:25], "\n")

Year 
 [2016] 

Country 
 ['Afghanistan' 'Albania' 'Algeria' 'Angola' 'Antigua and Barbuda'
 'Argentina' 'Armenia' 'Australia' 'Austria' 'Azerbaijan' 'Bahamas'
 'Bahrain' 'Bangladesh' 'Barbados' 'Belarus' 'Belgium' 'Belize' 'Benin'
 'Bhutan' 'Bolivia (Plurinational State of)' 'Bosnia and Herzegovina'
 'Botswana' 'Brazil' 'Brunei Darussalam' 'Bulgaria'] 

Granularity 
 ['Female' 'Male' 'Total'] 

Mortality rate attributed to exposure to unsafe WASH services 
 [15.31193 12.61297 13.92067  0.12552  0.2065   0.16641  2.1989   1.72837
  1.86723 45.15024 52.62506 48.81467  0.10419  0.12469  0.11403  0.40251
  0.32166  0.36294  0.16495  0.19032  0.17688  0.11958  0.07103  0.09539
  0.15099] 

WASH deaths 
 [         nan 4.824353e+03 4.869750e+00 7.582100e+02 1.406520e+04
 1.151300e-01 1.591411e+02 5.173360e+00 2.301346e+01 1.078108e+01
 1.089128e+02 2.681100e-01 5.643500e-01 1.946403e+04 4.691500e-01
 4.896670e+00 3.200647e+01 3.541880e+00 6.494034e+03 3.150916e+01
 6.072645e+02 2.768530e+00 

## Exploration

In [48]:
# La fonction est visible dans le fichier mylib
exploration1(mortal)

,Year,Country,Granularity,Mortality rate attributed to exposure to unsafe WASH services,WASH deaths
0,2016,Afghanistan,Female,15.311930,nan
1,2016,Afghanistan,Male,12.612970,nan
2,2016,Afghanistan,Total,13.920670,4824.353000
,Year,Country,Granularity,Mortality rate attributed to exposure to unsafe WASH services,WASH deaths
546,2016,Zimbabwe,Female,22.163880,nan
547,2016,Zimbabwe,Male,27.066880,nan
548,2016,Zimbabwe,Total,24.550740,3965.033000
,Year,Country,Granularity,Mortality rate attributed to exposure to unsafe WASH services,WASH deaths
462,2016,Sri Lanka,Female,1.199320,nan
391,2016,Qatar,Male,0.006290,nan


In [49]:
# La fonction est visible dans le fichier mylib
exploration2(mortal)

,Type des données,Nombre de valeurs totales,Nombre de NaN,NaN (%),Nombre de valeurs uniques,Valeurs uniques (%)
Year,int64,549,0,0.0,1,0.182149
Country,object,549,0,0.0,183,33.333333
Granularity,object,549,0,0.0,3,0.546448
Mortality rate attributed to exposure to unsafe WASH services,float64,549,0,0.0,548,99.817851
WASH deaths,float64,183,366,66.666667,183,33.333333


In [50]:
# La fonction est visible dans le fichier mylib
exploration3(mortal)


---------------------------------------
Nombre de lignes et de colonnes (shape)
--------------------------------------- 

(549, 5) 

--------------------------------------
Affichage des NaN (isna().any(axis=1))
-------------------------------------- 

     Year      Country Granularity  \
0    2016  Afghanistan      Female   
1    2016  Afghanistan        Male   
3    2016      Albania      Female   
4    2016      Albania        Male   
6    2016      Algeria      Female   
..    ...          ...         ...   
541  2016        Yemen        Male   
543  2016       Zambia      Female   
544  2016       Zambia        Male   
546  2016     Zimbabwe      Female   
547  2016     Zimbabwe        Male   

     Mortality rate attributed to exposure to unsafe WASH services  \
0                                             15.31193               
1                                             12.61297               
3                                              0.12552               
4         

In [51]:
# La fonction est visible dans le fichier mylib
exploration4(mortal)

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
Year,549.0,NaN,NaN,NaN,2016.0,0.0,2016.0,2016.0,2016.0,2016.0,2016.0
Country,549,183,Afghanistan,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Granularity,549,3,Female,183,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Mortality rate attributed to exposure to unsafe WASH services,549.0,NaN,NaN,NaN,12.493876,20.830508,0.00396,0.19296,1.28871,18.05478,107.04802
WASH deaths,183.0,NaN,NaN,NaN,4756.097706,21280.125369,0.08229,11.163275,130.9834,1950.4335,246087.9


In [52]:
# Renommage de colonne
mortal.rename(columns={'Granularity': 'Granularity mortality gender'}, inplace=True)

## Vérifions l'exactitude du nom des pays

In [53]:
mortal_countries = mortal['Country'].unique()
country_countries = country['Country'].unique()

# Compare les valeurs uniques des deux DataFrames
common_countries = set(mortal_countries) & set(country_countries)
unique_countries_in_mortal = set(mortal_countries) - set(country_countries)
unique_countries_in_country = set(country_countries) - set(mortal_countries)

# Affiche les résultats
print(bold + "Pays communs aux deux DataFrames :")
print("Nombre :", len(common_countries), end)
print(common_countries)
print(bold + "\nPays uniquement dans le DataFrame mortal :")
print("Nombre :", len(unique_countries_in_mortal), end)
print(unique_countries_in_mortal)
print(bold + "\nPays uniquement dans le DataFrame country :")
print("Nombre :", len(unique_countries_in_country), end)
print(unique_countries_in_country)

Pays communs aux deux DataFrames :
Nombre : 183 
{'Belarus', 'Kyrgyzstan', 'France', 'Republic of North Macedonia', 'Fiji', 'Honduras', 'Austria', 'Portugal', 'Liberia', 'Armenia', 'Estonia', 'Georgia', 'Russian Federation', 'Malawi', 'Sri Lanka', 'Syrian Arab Republic', 'Yemen', 'Italy', 'Albania', 'China', 'United States of America', 'Libya', 'Equatorial Guinea', 'Philippines', 'Latvia', 'Qatar', 'Samoa', 'Zambia', 'Cyprus', 'Lithuania', 'Lebanon', 'Iceland', 'Viet Nam', 'Indonesia', 'Thailand', 'Central African Republic', 'Djibouti', 'Senegal', 'Spain', 'Solomon Islands', 'Denmark', 'Ecuador', 'Greece', 'Namibia', 'Montenegro', 'Bulgaria', "Democratic People's Republic of Korea", 'Lesotho', 'Nigeria', 'Sierra Leone', 'Saint Lucia', 'Iraq', 'Cambodia', 'Mexico', 'Peru', 'Bahrain', 'Czechia', 'Pakistan', 'Suriname', 'Panama', 'Ukraine', 'Haiti', 'Guinea-Bissau', 'Oman', 'Nepal', 'Republic of Moldova', 'Comoros', 'Somalia', 'Japan', 'United Republic of Tanzania', 'Ireland', 'Eswatini',

**=> Pas de corrections à faire, pour la jointure il est important que tous les pays du DataFrame 'mortal' soient présents dans le DataFrame 'country'**

In [54]:
# Obtiens les valeurs uniques de la colonne 'Country' dans le premier DataFrame (pop)
mortal_countries = mortal['Country'].unique()

# Obtiens les valeurs uniques de la colonne 'Country' dans le deuxième DataFrame (country)
country_countries = country['Country'].unique()

# Définit un seuil de correspondance (score minimal pour considérer les pays comme correspondants)
threshold = 80

# Compare les noms de pays entre les deux DataFrames
matching_countries = []
for country_mortal in mortal_countries:
    best_match = process.extractOne(country_mortal, country_countries, scorer=fuzz.token_set_ratio)
    if best_match[1] >= threshold and country_mortal != best_match[0]:
        matching_countries.append((country_mortal, best_match[0]))

# Affiche les résultats
print(bold + "Correspondances entre les noms de pays (excluant les identiques) :" + end)
for country_pair in matching_countries:
    print("Pop : {}, Country : {}".format(country_pair[0], country_pair[1]))

Correspondances entre les noms de pays (excluant les identiques) :
Pop : Democratic Republic of the Congo, Country : Congo
Pop : Guinea, Country : Equatorial Guinea
Pop : Guinea-Bissau, Country : Guinea
Pop : Papua New Guinea, Country : Guinea
Pop : Republic of Korea, Country : Democratic People's Republic of Korea
Pop : Sudan, Country : South Sudan
Pop : United Kingdom of Great Britain and Northern Ireland, Country : Ireland


**=> Il s'agit bien de pays différents, il n'y a pas de correction à faire**

# **Political stabillity**

## Observations uniques

In [55]:
for i in stab.columns:
    print(bold + i + end, "\n", stab[i].unique()[0:25], "\n")

Country 
 ['Afghanistan' 'Albania' 'Algeria' 'American Samoa' 'Andorra' 'Angola'
 'Antigua and Barbuda' 'Argentina' 'Armenia' 'Australia' 'Austria'
 'Azerbaijan' 'Bahamas' 'Bahrain' 'Bangladesh' 'Barbados' 'Belarus'
 'Belgium' 'Belize' 'Benin' 'Bermuda' 'Bhutan'
 'Bolivia (Plurinational State of)' 'Bosnia and Herzegovina' 'Botswana'] 

Year 
 [2000 2002 2003 2004 2005 2006 2007 2008 2009 2010 2011 2012 2013 2014
 2015 2016 2017 2018] 

Political_Stability 
 [-2.44 -2.04 -2.2  -2.3  -2.07 -2.22 -2.41 -2.69 -2.71 -2.58 -2.5  -2.42
 -2.52 -2.57 -2.67 -2.8  -2.75 -0.54 -0.29 -0.31 -0.43 -0.51 -0.2  -0.03
 -0.05] 

Granularity 
 ['Total'] 



## Exploration

In [56]:
# La fonction est visible dans le fichier mylib
exploration1(stab)

,Country,Year,Political_Stability,Granularity
0,Afghanistan,2000,-2.440000,Total
1,Afghanistan,2002,-2.040000,Total
2,Afghanistan,2003,-2.200000,Total
,Country,Year,Political_Stability,Granularity
3523,Zimbabwe,2016,-0.620000,Total
3524,Zimbabwe,2017,-0.710000,Total
3525,Zimbabwe,2018,-0.710000,Total
,Country,Year,Political_Stability,Granularity
1316,Greenland,2012,1.930000,Total
1159,Fiji,2009,-0.270000,Total


In [57]:
# La fonction est visible dans le fichier mylib
exploration2(stab)

,Type des données,Nombre de valeurs totales,Nombre de NaN,NaN (%),Nombre de valeurs uniques,Valeurs uniques (%)
Country,object,3526,0,0.0,200,5.67215
Year,int64,3526,0,0.0,18,0.510493
Political_Stability,float64,3526,0,0.0,444,12.592172
Granularity,object,3526,0,0.0,1,0.028361


In [58]:
# La fonction est visible dans le fichier mylib
exploration3(stab)


---------------------------------------
Nombre de lignes et de colonnes (shape)
--------------------------------------- 

(3526, 4) 

--------------------------------------
Affichage des NaN (isna().any(axis=1))
-------------------------------------- 

Empty DataFrame
Columns: [Country, Year, Political_Stability, Granularity]
Index: [] 

---------------------------------------
Nombre de doublons (duplicated().sum())
--------------------------------------- 

0 

-----------------------------------------------------
Affichage des doublons (df[df.duplicated()].head(10))
----------------------------------------------------- 

Empty DataFrame
Columns: [Country, Year, Political_Stability, Granularity]
Index: [] 



In [59]:
# La fonction est visible dans le fichier mylib
exploration4(stab)

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
Country,3526,200,Afghanistan,18,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Year,3526.0,NaN,NaN,NaN,2009.521838,5.255833,2000.0,2005.0,2010.0,2014.0,2018.0
Political_Stability,3526.0,NaN,NaN,NaN,-0.051044,0.996039,-3.31,-0.71,0.05,0.7975,1.97
Granularity,3526,1,Total,3526,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [60]:
# Suppression de colonne non pertinente pour la suite de l'étude
stab.drop(columns='Granularity', inplace=True)

In [61]:
stab.sample()

,Country,Year,Political_Stability
2100,Mongolia,2010,0.6


## Vérifions l'exactitude du nom des pays

In [62]:
stab_countries = stab['Country'].unique()
country_countries = country['Country'].unique()

# Compare les valeurs uniques des deux DataFrames
common_countries = set(stab_countries) & set(country_countries)
unique_countries_in_stab = set(stab_countries) - set(country_countries)
unique_countries_in_country = set(country_countries) - set(stab_countries)

# Affiche les résultats
print(bold + "Pays communs aux deux DataFrames :")
print("Nombre :", len(common_countries), end)
print(common_countries)
print(bold + "\nPays uniquement dans le DataFrame stab :")
print("Nombre :", len(unique_countries_in_stab), end)
print(unique_countries_in_stab)
print(bold + "\nPays uniquement dans le DataFrame country :")
print("Nombre :", len(unique_countries_in_country), end)
print(unique_countries_in_country)

Pays communs aux deux DataFrames :
Nombre : 200 
{'Belarus', 'Kyrgyzstan', 'France', 'Republic of North Macedonia', 'Fiji', 'Honduras', 'Austria', 'Portugal', 'Liberia', 'Armenia', 'Estonia', 'Georgia', 'Russian Federation', 'Malawi', 'Sri Lanka', 'Syrian Arab Republic', 'Yemen', 'Italy', 'Albania', 'China', 'Puerto Rico', 'United States of America', 'Libya', 'Equatorial Guinea', 'Philippines', 'Latvia', 'Niue', 'Qatar', 'Samoa', 'Zambia', 'Cyprus', 'Lithuania', 'Lebanon', 'Iceland', 'Viet Nam', 'Indonesia', 'Thailand', 'Central African Republic', 'Djibouti', 'Senegal', 'Spain', 'Solomon Islands', 'Denmark', 'Ecuador', 'Greece', 'Namibia', 'Montenegro', 'Bulgaria', "Democratic People's Republic of Korea", 'Lesotho', 'Nigeria', 'Sierra Leone', 'Saint Lucia', 'Iraq', 'Cambodia', 'Mexico', 'Peru', 'Bahrain', 'Czechia', 'Pakistan', 'Suriname', 'Panama', 'Ukraine', 'Haiti', 'Guinea-Bissau', 'Oman', 'Nepal', 'Republic of Moldova', 'Comoros', 'Somalia', 'Japan', 'United Republic of Tanzania',

**=> Pas de corrections à faire, pour la jointure il est important que tous les pays du DataFrame 'stab' soient présents dans le DataFrame 'country'**

In [63]:
# Obtiens les valeurs uniques de la colonne 'Country' dans le premier DataFrame (pop)
stab_countries = stab['Country'].unique()

# Obtiens les valeurs uniques de la colonne 'Country' dans le deuxième DataFrame (country)
country_countries = country['Country'].unique()

# Définit un seuil de correspondance (score minimal pour considérer les pays comme correspondants)
threshold = 80

# Compare les noms de pays entre les deux DataFrames
matching_countries = []
for country_stab in stab_countries:
    best_match = process.extractOne(country_stab, country_countries, scorer=fuzz.token_set_ratio)
    if best_match[1] >= threshold and country_stab != best_match[0]:
        matching_countries.append((country_stab, best_match[0]))

# Affiche les résultats
print(bold + "Correspondances entre les noms de pays (excluant les identiques) :" + end)
for country_pair in matching_countries:
    print("Pop : {}, Country : {}".format(country_pair[0], country_pair[1]))

Correspondances entre les noms de pays (excluant les identiques) :
Pop : American Samoa, Country : Samoa
Pop : Democratic Republic of the Congo, Country : Congo
Pop : Guinea, Country : Equatorial Guinea
Pop : Guinea-Bissau, Country : Guinea
Pop : Papua New Guinea, Country : Guinea
Pop : Republic of Korea, Country : Democratic People's Republic of Korea
Pop : Sudan, Country : South Sudan
Pop : United Kingdom of Great Britain and Northern Ireland, Country : Ireland


**=> Il s'agit bien de pays différents, il n'y a pas de correction à faire**

# **Exportation des fichiers**

In [64]:
country.to_excel('src/country_cleaned.xlsx')
pop.to_excel('src/pop_cleaned.xlsx')
water.to_excel('src/water_cleaned.xlsx')
stab.to_excel('src/stab_cleaned.xlsx')
mortal.to_excel('src/mortal_cleaned.xlsx')